In [1]:
import wave
import struct

wavefile=wave.open('3mm_decimate4.wav','r')
nchannel=wavefile.getnchannels()
length=wavefile.getnframes()
print(nchannel)
print(length)
print(wavefile.getframerate())
print(wavefile.getsampwidth())

sampleRate = wavefile.getframerate() # hertz
duration = 1.0       # seconds
frequency = 440.0    # hertz

nread=length
if nread<=length:
    nbytes=nchannel*nread
    format="<" + str(nbytes) + "h"
    wavedata=wavefile.readframes(nread)
    data_a=struct.unpack(format,wavedata)
wavefile.close()

if nchannel==1:
    wave_channel1=list(data_a)
elif nchannel==2:
    wave_channel1=list(data_a[0::2])
    wave_channel2=list(data_a[1::2])



1
739602
4000
2


In [2]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


seq_length = 8000
dataX = []
dataY = []
print('1phase done')
for i in range(0, (length - seq_length),2000):
    seq_in = wave_channel1[i:i + seq_length]
    dataX.append( seq_in)
    dataY.append(i)
print('2phase done')  
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / 32768.0


Using TensorFlow backend.


1phase done
2phase done
Total Patterns:  366


In [ ]:
# one hot encode the output variable
samplepos=length/256;
datY=[(int)((dat)/samplepos) for dat in dataY]
y = np_utils.to_categorical(datY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=4, callbacks=callbacks_list)

Epoch 1/50
216/366 [================>.............] - ETA: 1175s - loss: 5.5769

In [ ]:
wavefile=wave.open('seed.wav','r')
nchannel=wavefile.getnchannels()
slength=wavefile.getnframes()
print(nchannel)
print(slength)
print(wavefile.getframerate())
print(wavefile.getsampwidth())

sampleRate = wavefile.getframerate() # hertz
duration = 1.0       # seconds
frequency = 440.0    # hertz

sread=slength
if sread<=length:
    nbytes=nchannel*sread
    format="<" + str(nbytes) + "h"
    wavedata=wavefile.readframes(sread)
    data_a=struct.unpack(format,wavedata)
wavefile.close()

if nchannel==1:
    seed1=list(data_a)
elif nchannel==2:
    seed1=list(data_a[0::2])
    seed2=list(data_a[1::2])


In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-30-1.7248-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
pattern = seed1[0:8000]
print("Seed:")400-
# generate characters
x = numpy.reshape(pattern, (1, len(pattern), 1))
x = x / float(n_vocab)
prediction = model.predict(x, verbose=0)
index = numpy.argmax(prediction)
print("위치 : "+(int)(index*samplepos/4000)+"초") 
print("\nDone.")